In [ ]:
import holoviews as hv; hv.extension('bokeh', 'plotly', logo=None)
import panel as pn;     pn.extension()
import numpy as np
import os
import scipy.io

In [ ]:
dbg=False
if dbg:
    import matplotlib.pyplot as plt
    import numpy as np
    import os
    import scipy.io
    plt.rcParams['figure.figsize'] = [8, 8]
    plt.rcParams.update({'font.size': 18})

In [ ]:
mat_contents = scipy.io.loadmat(os.path.join('..','DATA','allFaces.mat'))
faces        = mat_contents['faces']
m            = int(mat_contents['m'])
n            = int(mat_contents['n'])
nfaces       = np.ndarray.flatten(mat_contents['nfaces'])

# We use the first 36 people for training data
trainingFaces = faces[:,:np.sum(nfaces[:36])]
avgFace       = np.mean(trainingFaces,axis=1) # size n*m by 1

# Compute eigenfaces on mean-subtracted training data
X        = trainingFaces - np.tile(avgFace,(trainingFaces.shape[1],1)).T
U, S, VT = np.linalg.svd(X,full_matrices=0)

In [ ]:
pn.interact( lambda i=0: (hv.Raster( np.reshape(avgFace,(m,n)).T ).opts(title='Average Face')+\
                        hv.Raster( np.reshape(U[:,i],(m,n)).T  ).opts(title=f'Eigenface {i}'))\
                       .opts(hv.opts.Raster(cmap='gray', xaxis=None, yaxis=None, frame_width=150, aspect='equal')).opts( shared_axes=False ),
            i = (0,U.shape[1]-1)
           )

In [ ]:
## Now show eigenface reconstruction of image that was omitted from test set

testFace   = faces[:,np.sum(nfaces[:36])] # First face of person 37
testFaceMS = testFace - avgFace

def reconstruct(r):
    print( '  ', r)
    reconFace = avgFace + U[:,:r]  @ U[:,:r].T @ testFaceMS
    return np.reshape(reconFace,(m,n)).T

r_list   = [25, 50, 100, 200, 400, 800, 1600]
img_data = [reconstruct(r) for r in r_list]

In [ ]:
h_opts     = hv.opts.Raster( cmap = 'gray', xaxis = None, yaxis = None, frame_width = 150, aspect = 'equal')
h_testFace = hv.Raster( np.reshape(avgFace,(m,n)).T ).opts(title='Original Image')

#hv.DynamicMap( lambda r:( h_testFace+reconstruct(r)), kdims=['r'] ).redim.values(r=r_list)
pn.interact(lambda i: ( h_testFace+hv.Raster(img_data[i]).opts(title=f"r={r_list[i]}"))\
                      .opts(shared_axes=False).opts(h_opts), i=(0,len(r_list)-1))

In [ ]:
## Now show eigenface reconstruction of image that was omitted from test set
if False:
    testFace = faces[:,np.sum(nfaces[:36])] # First face of person 37
    plt.imshow(np.reshape(testFace,(m,n)).T)
    plt.set_cmap('gray')
    plt.title('Original Image')
    plt.axis('off')
    plt.show()

    testFaceMS = testFace - avgFace
    r_list = [25, 50, 100, 200, 400, 800, 1600]

    for r in r_list:
        reconFace = avgFace + U[:,:r]  @ U[:,:r].T @ testFaceMS
        img       = plt.imshow(np.reshape(reconFace,(m,n)).T)
        img.set_cmap('gray')
        plt.title('r = ' + str(r))
        plt.axis('off')
        plt.show()

In [ ]:
## Project person 2 and 7 onto PC5 and PC6

P1num = 2 # Person number 2
P2num = 7 # Person number 7

P1 = faces[:,np.sum(nfaces[:(P1num-1)]):np.sum(nfaces[:P1num])]
P2 = faces[:,np.sum(nfaces[:(P2num-1)]):np.sum(nfaces[:P2num])]

P1 = P1 - np.tile(avgFace,(P1.shape[1],1)).T
P2 = P2 - np.tile(avgFace,(P2.shape[1],1)).T

PCAmodes = [5, 6] # Project onto PCA modes 5 and 6
PCACoordsP1 = U[:,PCAmodes-np.ones_like(PCAmodes)].T @ P1
PCACoordsP2 = U[:,PCAmodes-np.ones_like(PCAmodes)].T @ P2

In [ ]:
if dbg:
    plt.plot(PCACoordsP1[0,:],PCACoordsP1[1,:],'d',color='k',label='Person 2')
    plt.plot(PCACoordsP2[0,:],PCACoordsP2[1,:],'^',color='r',label='Person 7')

    plt.legend()
    plt.show()